In [79]:
import os
import requests
import json
import pandas as pd
import subprocess

from spellchecker import SpellChecker

In [80]:
prom_query_url = 'http://13.125.63.249:30915/api/v1/query?query='

In [81]:
def call_data(url, query):
    r = requests.get(url + query)
    return r

In [82]:
url_prom = 'http://13.125.63.249:30915/api/v1/query?query='
query = 'ALERTS{alertname="KubePodCrashLooping"}'
json.loads(call_data(url_prom, query).text)


{'status': 'success',
 'data': {'resultType': 'vector',
  'result': [{'metric': {'__name__': 'ALERTS',
     'alertname': 'KubePodCrashLooping',
     'alertstate': 'firing',
     'container': 'mysql',
     'endpoint': 'http',
     'instance': '10.220.105.27:8080',
     'job': 'kube-state-metrics',
     'namespace': 'test-alerts',
     'pod': 'test-mysql-01-6cc9b56f8b-5pk9x',
     'service': 'prom-kube-state-metrics',
     'severity': 'critical'},
    'value': [1600933031.168, '1']},
   {'metric': {'__name__': 'ALERTS',
     'alertname': 'KubePodCrashLooping',
     'alertstate': 'firing',
     'container': 'mysql',
     'endpoint': 'http',
     'instance': '10.220.105.27:8080',
     'job': 'kube-state-metrics',
     'namespace': 'test-alerts',
     'pod': 'test-mysql-aa-7c8575bcfc-k7n6x',
     'service': 'prom-kube-state-metrics',
     'severity': 'critical'},
    'value': [1600933031.168, '1']}]}}

In [83]:
def kube_pod_crash_looping():
    ## Alerts case : KubePodCrashLooping
    alert_query = 'ALERTS{alertname="KubePodCrashLooping"}'
    alert_get = call_data(prom_query_url, alert_query)
    alert_message = json.loads(alert_get.text)
    
    fix_list = []
    for item in alert_message['data']['result']:
        #print(item)
        check_list = {'namespace':item['metric']['namespace'], 'pod':item['metric']['pod']}
        #check_list = {item['metric']['namespace'], item['metric']['pod']}
        #print(check_list)        
        fix_list.append(check_list) 
    
    return(fix_list) 

In [84]:
kube_pod_crash_looping()

[{'namespace': 'test-alerts', 'pod': 'test-mysql-01-6cc9b56f8b-5pk9x'},
 {'namespace': 'test-alerts', 'pod': 'test-mysql-aa-7c8575bcfc-k7n6x'}]

In [85]:
def system_recommand():
    pending_data = kube_pod_crash_looping()
    cnt = len(pending_data)    
    df_data = pd.DataFrame(pending_data)
    
    kubectl_result = []
    for i in range(cnt):
        namespace = df_data.namespace[i]
        pod = df_data.pod[i]
        
        cli = "kubectl -n " + namespace + " logs " + pod
        kubectl_command_line = subprocess.check_output(cli, shell=True,  encoding='utf-8')
        
        print("아래 로그로 POD가 정상 동작하지 않습니다. ")
        print("네임스페이스 : " + namespace \
              + "\n" + "파드 : " + pod \
              + "\n" + "원인 : " + kubectl_command_line \
              + "\n" + "확인 명령어" \
              + "\n" + cli + "\n" + "##########"\
              + "\n") 
         
    #return(kubectl_result)
    

In [86]:
if __name__=="__main__":
    #print(kube_container_waiting())
    #print(kubectl_jsonpath())
    print(system_recommand())
    

아래 로그로 POD가 정상 동작하지 않습니다. 
네임스페이스 : test-alerts
파드 : test-mysql-01-6cc9b56f8b-5pk9x
원인 : 2020-09-24 07:34:26+00:00 [Note] [Entrypoint]: Entrypoint script for MySQL Server 8.0.21-1debian10 started.
2020-09-24 07:34:26+00:00 [Note] [Entrypoint]: Switching to dedicated user 'mysql'
2020-09-24 07:34:26+00:00 [Note] [Entrypoint]: Entrypoint script for MySQL Server 8.0.21-1debian10 started.
2020-09-24 07:34:26+00:00 [ERROR] [Entrypoint]: Database is uninitialized and password option is not specified
	You need to specify one of MYSQL_ROOT_PASSWORD, MYSQL_ALLOW_EMPTY_PASSWORD and MYSQL_RANDOM_ROOT_PASSWORD

확인 명령어
kubectl -n test-alerts logs test-mysql-01-6cc9b56f8b-5pk9x
##########

아래 로그로 POD가 정상 동작하지 않습니다. 
네임스페이스 : test-alerts
파드 : test-mysql-aa-7c8575bcfc-k7n6x
원인 : 2020-09-24 07:33:02+00:00 [Note] [Entrypoint]: Entrypoint script for MySQL Server 8.0.21-1debian10 started.
2020-09-24 07:33:02+00:00 [Note] [Entrypoint]: Switching to dedicated user 'mysql'
2020-09-24 07:33:02+00:00 [Note] [E